In [3]:
# this file contains all the code needed to run the model
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.cuda.amp import autocast
import timm
from typing import List, Optional
from timm.layers import SelectAdaptivePool2d, create_conv2d, GELUTanh
import itertools
import torchvision

In [1]:
# deit3_small_patch16_224

# swin_base_patch4_window7_224

# model = timm.create_model('resnet50', pretrained=True, num_classes=len(class_labels))
# print(model)

NameError: name 'timm' is not defined

In [7]:
def same_padding(kernel_size ) :
    if isinstance(kernel_size , tuple):
        return tuple([same_padding(s) for s in kernel_size])
    else :
        assert kernel_size % 2 != 0, "kernel size must me odd to make the padding equal across the image" #if padding is even we might need to add 2 differnt paddings across the width and bredth 
        return kernel_size // 2

In [13]:
def val2tuple(x: list or tuple or any, min_len: int = 1, idx_repeat: int = -1) :

    if isinstance(x, tuple):
       
        return x

    
    x = val2list(x)

    # repeat elements if necessary
    if len(x) > 0:
        x[idx_repeat:idx_repeat] = [x[idx_repeat] for _ in range(min_len - len(x))]

    return tuple(x)


def val2list(x: list or tuple or any, repeat_time=1) -> list:
    if isinstance(x, (list, tuple)):
        return list(x)
    return [x for _ in range(repeat_time)]

In [9]:
class ConvLayer(nn.Module):
    def __init__(self,
    in_channels : int,
    out_channels : int,
    kernel_size  = 3,
    stride = 1,
    dilation = 1,
    groups = 1,
    use_bias = True,
    dropout = 0 ,
    pad = None,
    norm  = None,
    activation = None,
    ):
        super(ConvLayer, self).__init__()
        pad = same_padding(kernel_size) if pad == None else pad
        pad *= dilation
        self.dropout = nn.Dropout2d(dropout, inplace=False) 


       
        # print("hello" , in_channels , out_channels , kernel_size , stride , pad , dilation , groups , use_bias)

        # Parameter(torch.empty((out_channels, in_channels // groups, *kernel_size), **factory_kwargs))

        
        self.Convlayer = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size= (kernel_size,kernel_size),
            stride=(stride, stride),
            padding=pad,
            dilation=(dilation, dilation),
            groups=groups,
            bias=use_bias,
        )
        # print("BYE")

        if norm == nn.LayerNorm :
            self.norm = nn.LayerNorm([8, 256, 14, 14])
        else:
        
            self.norm = norm(num_features=out_channels) if norm else nn.Identity()
        
        self.activation = activation(inplace = False) if activation is not None else nn.Identity()
       
        # normalization done last self.norm = 
        #activation done last 

    def forward(self , x):
        
        x = self.dropout(x)
        x = self.Convlayer(x)
   
        x = self.norm(x)
        
        x = self.activation(x)
        return x


In [10]:
x = torch.Tensor([[[1,2] , [1,2]] , [[1,2] , [3,4]] , [[0,1] , [2,3]]])

p = torch.mean(x, dim=1, keepdim=True)
x - p

tensor([[[ 0.,  0.],
         [ 0.,  0.]],

        [[-1., -1.],
         [ 1.,  1.]],

        [[-1., -1.],
         [ 1.,  1.]]])

In [1]:
parameters_dict_DWConv = {
    'bias' : (True , True),
    'norm' : (None , None),
    'activation' : (None , nn.ReLU)
}


class DepthWiseSeperableConvLayer(nn.Module):
    def __init__(self,
    in_channels,
    out_channels,
    kernel_size  = 3,
    stride = 1,
    use_bias  = parameters_dict_DWConv['bias'],
    norm  = parameters_dict_DWConv['norm'],
    activation  = parameters_dict_DWConv['activation']):

        

        super(DepthWiseSeperableConvLayer, self).__init__()

        use_bias = val2tuple(use_bias, 2)
        norm = val2tuple(norm, 2)
        activation = val2tuple(activation, 2)

        self.Splitting = ConvLayer(
            in_channels,
            in_channels,
            kernel_size= kernel_size,
            stride= stride,
            groups=in_channels,
            norm = norm[0],
            activation = activation[0],
            use_bias=use_bias[0],
        )
        self.Combining = ConvLayer(
            in_channels,
            out_channels,
            kernel_size=1,
            stride=stride,
            groups=1,
            norm = norm[1],
            activation = activation[1],
            use_bias=use_bias[1],
        )
    def forward(self , x : torch.Tensor):
        x = self.Splitting(x)
        return self.Combining(x)

NameError: name 'nn' is not defined

In [16]:
parameters_dict_MBConv = {
    'bias' : (True , True , False),
    'norm' : (None , None , nn.BatchNorm2d),
    'activation' : (nn.ReLU6 , nn.ReLU6 , None)
}

class MobileBottleneckConvLayer(nn.Module):
    def __init__(self , 
    in_channels : int,
    out_channels : int,
    stride = 1,
    kernel_size = 3,
    groups = 1,
    expand_ratio = 5,
    use_bias  = parameters_dict_MBConv['bias'],
    norm  = parameters_dict_MBConv['norm'],
    activation  = parameters_dict_MBConv['activation']):

        super(MobileBottleneckConvLayer, self).__init__()

        use_bias = val2tuple(use_bias, 3)
        norm = val2tuple(norm, 3)
        activation = val2tuple(activation, 3)

        

        mid_channel = round(in_channels*expand_ratio)
        self.Expand = ConvLayer(
                in_channels,
                mid_channel,
                kernel_size=1,
                stride=1,
                norm = norm[0],
                activation = activation[0],
                use_bias=use_bias[0],
                groups = groups
            
                
            )
        self.Splitting = ConvLayer(
                mid_channel,
                mid_channel,
                kernel_size=kernel_size,
                stride=stride,
                groups=mid_channel,
                norm = norm[1],
                activation = activation[1],
                use_bias=use_bias[1]
                
            )

        
        self.Project = ConvLayer(
                mid_channel,
                out_channels,
                kernel_size=1,
                stride=1,
                groups = groups,
                norm = norm[2],
                activation = activation[2],
                use_bias=use_bias[2]
            )

    def forward(self , x : torch.Tensor):
        x = self.Expand(x)
        x = self.Splitting(x)
        return self.Project(x)

In [15]:
parameters_dict_FMBConv = {
    'bias' : (True , True),
    'norm' : (nn.BatchNorm2d , nn.BatchNorm2d ),
    'activation' : (nn.ReLU , nn.ReLU )
}

class FusedMobileBottleneckConvLayer(nn.Module):
    def __init__(
        self,
        in_channels: int,
        out_channels: int,
        kernel_size=3,
        stride=1,
        mid_channels=None,
        expand_ratio=6,
        groups=1,
        use_bias=False,
        norm=(nn.BatchNorm2d, nn.BatchNorm2d),
        activation=('ReLU6', None),
    ):
        super(FusedMobileBottleneckConvLayer, self).__init__()
        use_bias = val2tuple(use_bias, 2)

        
        norm = val2tuple(norm, 2)

        if not isinstance(x, tuple):
            
            activation = val2tuple(activation, 2)
            
        mid_channels = mid_channels or round(in_channels * expand_ratio)

       

        self.spatial_conv = ConvLayer(
            in_channels,
            mid_channels,
            kernel_size,
            stride=stride,
            groups=groups,
            norm=norm[0],
            activation=activation[0],
            use_bias=use_bias[0],
        )
        self.point_conv = ConvLayer(
            mid_channels,
            out_channels,
            1,
            norm=norm[1],
            activation=activation[1],
            use_bias=use_bias[1],
        )

    def forward(self, x):
        x = self.spatial_conv(x)
        x = self.point_conv(x)
        return x

# class FusedMobileBottleneckConvLayer(nn.Module):
#     def __init__(self , 
#     in_channels : int,
#     out_channels : int,
#     stride = 1,
#     kernel_size = 3,
#     groups = 1,
#     expand_ratio = 5,
#     use_bias : tuple[bool,...] = parameters_dict_FMBConv['bias'],
#     norm : tuple[str,...] = parameters_dict_FMBConv['norm'],
#     activation : tuple[str,...] = parameters_dict_FMBConv['activation']):
        
#         super(FusedMobileBottleneckConvLayer, self).__init__()

#         use_bias = val2tuple(use_bias, 2)
#         norm = val2tuple(norm, 2)
#         activation = val2tuple(activation, 2)
        
#         mid_channel = round(in_channels*expand_ratio)


#         print(norm,activation,use_bias)
#         self.Expand_Depth = ConvLayer(
#                 in_channels,
#                 mid_channel,
#                 kernel_size=kernel_size,
#                 stride=stride,
#                 groups = groups,
#                 norm = norm[0],
#                 activation = activation[0],
#                 use_bias=use_bias[0],
#             )
        

#         self.Project = ConvLayer(
#                 mid_channel,
#                 out_channels,
#                 kernel_size=1,
#                 stride=(stride, stride),
#                 groups = groups,
#                 norm = norm[1],
#                 activation = activation[1],
#                 use_bias=use_bias[1]
#             )

#     def forward(self , x) :
#         x = self.Expand_Depth(x)
#         print(x.shape )
#         x = self.Project(x)
#         return x

In [73]:
parameters_dict_ResB = {
    'bias' : (True , True , True),
    'norm' : (nn.BatchNorm2d , nn.BatchNorm2d , nn.BatchNorm2d),
    'activation' : (nn.ReLU , nn.ReLU , nn.ReLU)
}

class ResBlock(nn.Module):
    def __init__(self , 
    in_channels : int,
    out_channels : int,
    stride = 1,
    kernel_size = 3,
    groups = 1,
    expand_ratio = 1,
    use_bias  = parameters_dict_ResB['bias'],
    norm  = parameters_dict_ResB['norm'],
    activation  = parameters_dict_ResB['activation']):
        
        super(ResBlock, self).__init__()
        # print(use_bias)

        use_bias = val2tuple(use_bias, 2)
        norm = val2tuple(norm, 2)
        activation = val2tuple(activation, 2)
        
        mid_channel = round(in_channels*expand_ratio)
        self.Conv1 = ConvLayer(
                in_channels,
                mid_channel,
                kernel_size=kernel_size,
                stride=stride,
            
                groups = groups,
                norm = norm[0],
                activation = activation[0],
                use_bias=use_bias[0],
            )
        self.Conv2 = ConvLayer(
                mid_channel,
                out_channels,
                kernel_size=kernel_size,
                stride=stride,
             
                groups = groups,
                norm = norm[1],
                activation = activation[1],
                use_bias=use_bias[1]
            )
    def forward(self , x : torch.Tensor):
        first = self.Conv1(x)
        second =self.Conv2(first)
    
        return second

In [104]:
class LinearLayer(nn.Module):
    def __init__(self,
    input_shape : int,
    output_shape : int,
    dropout = 0.3,
    use_bias = True,
    norm = nn.LayerNorm,
    activation = GELUTanh):
        super(LinearLayer, self).__init__()
        self.dropout = nn.Dropout2d(dropout, inplace=False) if dropout > 0 else None
        self.linear = nn.Linear(input_shape ,output_shape , bias = use_bias)
        
        #norm and #activation done later 
        self.norm = norm(output_shape)
        self.activation = activation
        
    def squeeze(self , x : torch.tensor):
        if x.item().dim() > 2 :
            x = torch.flatten(x, start_dim=1)
        return x

    def forward(self , x : torch.tensor):
        
        x = self.squeeze(x)
        if self.dropout is not None :
            x = self.dropout(x)
        x = self.linear(x)
        if self.norm:
            x = self.norm(x)
        if self.activation:
            x = self.activation(x)
        return x


## My mod

In [1]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)


NameError: name 'torch' is not defined

NameError: name 'nn' is not defined

In [2]:
class IdentityLayer(nn.Module):
    def forward(self , x : torch.Tensor):
        return x


NameError: name 'nn' is not defined

In [97]:
# class LiteMLA(nn.Module):
#     def __init__(self , 
#     in_channels,
#     out_channels,
#     heads = None,
#     heads_ratio = 1.0,
#     dimention=8,
#     use_bias=False,
#     norm=(None, nn.BatchNorm2d),
#     activation=(None, None),
#     kernel_func=nn.ReLU,
#     scales: tuple[int, ...] = (5,),
#     eps=1.0e-15,
#     ):

        
#         super(LiteMLA, self).__init__()
#         heads = heads or (in_channels//(dimention * heads_ratio))
#         total_dimentions = int(dimention * heads)
#         self.eps = eps

#         use_bias = val2tuple(use_bias, 2)
#         norm = val2tuple(norm, 2)
#         activation = val2tuple(activation, 2)
        
#         self.dim = dimention
#         # print(norm)
#         # print(in_channels , int(3*total_dimentions) , norm[0],None,use_bias)
#         self.qkv = ConvLayer(
#                 in_channels = in_channels,
#                 out_channels = 3 * total_dimentions,
#                 kernel_size=1,
                
#                 norm = norm[0],
#                 activation = activation[0],
#                 use_bias=use_bias[0],
#             )

#         print(scales)
#         self.aggregate = nn.ModuleList([
#             nn.Sequential(
#                 ConvLayer(
#                 3 * total_dimentions,
#                 3 * total_dimentions,
#                 kernel_size=scale,
#                 groups = 3 * total_dimentions,
                
#                 use_bias=use_bias[0],
#             ),
#                 ConvLayer(
#                 3 * total_dimentions,
#                 3 * total_dimentions,
#                 kernel_size=1,
#                 groups = int(3 * heads),
                
#                 use_bias=use_bias[0],
#             )  
                   
#             )
#             for scale in scales
#         ])

#         self.activation = kernel_func(inplace=False)
#         self.projection = ConvLayer(
#                 total_dimentions * (1 + len(scales)),
#                 out_channels,
#                 kernel_size=1,
                
#                 norm = norm[1],
#                 activation = activation[1],
#                 use_bias=use_bias[1],
#             )  
#     @autocast(enabled=False)
#     def ReLU_linear_activation(self,
#                               qkv : torch.Tensor):
#         B, _, H, W = list(qkv.size())

#         if qkv.dtype == torch.float16:
#             qkv = qkv.float()

#         qkv_reshaped = torch.reshape(qkv , (B,-1,3*self.dim,H*W))
#         q, k, v = (
#             qkv[:, :, 0 : self.dim],
#             qkv[:, :, self.dim : 2 * self.dim],
#             qkv[:, :, 2 * self.dim :],
#         )
#         q = self.activation(q)
#         k = self.activation(k)
        
#         trans_k = k.transpose(-1, -2)

#         v = F.pad(v, (0, 0, 0, 1), mode="constant", value=1)
#         vk = torch.matmul(v, trans_k)
#         out = torch.matmul(vk, q)
#         if out.dtype == torch.bfloat16:
#             out = out.float()
#         out = out[:, :, :-1] / (out[:, :, -1:] + self.eps)

#         out = torch.reshape(out, (B, -1, H, W))
#         return out

#     def forward(self , x : torch.Tensor):
#         qkv = self.qkv(x)
#         multi_scale_qkv = [qkv]
#         for op in self.aggregate:
#             multi_scale_qkv.append(op(qkv))
#         qkv = torch.cat(multi_scale_qkv, dim=1)
#         out = self.ReLU_linear_activation(qkv)
#         out = self.projection(out)

#         return out







In [ ]:
class EfficientViTModule(nn.Module):
    def __init__(self,
                in_channels,
                dimention=32,
                heads_ratio= 1.0,
                expand_ratio: float = 4,
                scales=(5,),
                norm=nn.BatchNorm2d,
                activation="hswish",
            ):
        super(EfficientViTModule, self).__init__()
        self.context_module = ResidualBlock(
            LiteMLA(
            in_channels = in_channels,
            out_channels = in_channels,
            dimention = dimention,
            heads_ratio = heads_ratio,
            scales = scales,
            norm=(None , norm)), IdentityLayer())
        
        local_mod = MobileBottleneckConvLayer(
            in_channels=in_channels,
            out_channels=in_channels,
            expand_ratio=expand_ratio,
            use_bias=(True, True, False),
            norm=(None, None, norm),
            activation=(activation, activation, None),
        )
        self.local_module = ResidualBlock(local_mod, IdentityLayer())

    def forward(self , x:torch.Tensor):
        x = self.context_module(x)
        x = self.local_module(x)
        return x


In [1]:
class ResidualBlock(nn.Module):
    def __init__(
        self,
        main: Optional[nn.Module],
        shortcut: Optional[nn.Module] = None,
        pre_norm: Optional[nn.Module] = None,
    ):
        super(ResidualBlock, self).__init__()
        self.pre_norm = pre_norm if pre_norm is not None else nn.Identity()
        self.main = main
        self.shortcut = shortcut

    def forward(self, x):
        res = self.main(self.pre_norm(x))
        if self.shortcut is not None:
            # # print(x.shape)
            # print(res.shape)
            short = self.shortcut(x)
            # print("short" , short.shape)
            res = res + short
            
        return res

NameError: name 'nn' is not defined

In [1]:
# class OpSequential(nn.Module):
#     def __init__(self, op_list: list[nn.Module or None]):
#         super(OpSequential, self).__init__()
#         valid_op_list = []
#         for op in op_list:
#             if op is not None:
#                 valid_op_list.append(op)
#         self.op_list = nn.ModuleList(valid_op_list)

#     def forward(self, x: torch.Tensor) -> torch.Tensor:
#         for op in self.op_list:
#             x = op(x)
#         return x

In [80]:
# activation funciton 
dict = {
    'LeakyReLU':nn.LeakyReLU,
    'Hardswish':nn.Hardswish,
    nn.ReLU:nn.ReLU,
    'ReLU6':nn.ReLU6,
    GELUTanh:GELUTanh,
    'Sigmoid':nn.Sigmoid,
    'SiLU':nn.SiLU,
}

def act_fun( x : str) -> nn.Module:
    if x in dict:
        print('ehy' , x)
        return dict[x]
    else:
        return nn.Identity()

In [81]:
#normalization 

dict_norm = {
    nn.BatchNorm2d : nn.BatchNorm2d,
    nn.LayerNorm : nn.LayerNorm
}

def norm_fun( x : str):
    if x in dict_norm:
        
        return dict_norm[x]
    else:
        return nn.Identity()


In [2]:
# # ## Backbone 

# class MyModule(nn.Module):
#     '''
    
#     '''
    
#     def __init__(self,
#             in_channels,
#             out_channels,
#             heads = 10,
#             dim = 32,
#             ):
        
#         super(MyModule, self).__init__()
#         self.heads = heads
#         self.dim = dim
#         scale = dim
#         self.scale = dim ** -0.5

#         # all the 3 conv
#         self.total_layer = 4

      
        
        
#         self.get_qkv = ConvLayer(in_channels , self.dim * 3  , 3 , norm = nn.BatchNorm2d)

        
        
       
#         self.local_mod = MobileBottleneckConvLayer(out_channels , out_channels , 3 , norm = nn.BatchNorm2d)
       

        
#         self.proj =  ConvLayer(self.dim  , out_channels , 3 , norm = nn.BatchNorm2d)
        
#         # self.multilpier = ConvLayer(self.dim  , out_channels , 3 , norm = nn.BatchNorm2d)
        
    
        

#     def attention(self,q,k,v):

        
        
#         q, k, v = q.float(), k.float(), v.float()

#         print("1" , q.shape , k.shape , v.shape)

#         q = q * self.scale
#         att_map = q.transpose(-2, -1) @ k

       
#         att_map = att_map.softmax(dim=-1)

       
#         out = v @ att_map.transpose(-2, -1)

    
        
#         return out

#     def forward(self,x):
#         B, C, H, W = x.shape


        
#             # print(op)

            

#         qkv  = self.get_qkv(x)
    
#         qkv = qkv.reshape(B, -1, 3 * self.dim, H * W).transpose(-1, -2)
#         print(qkv.shape)
        
#         q, k, v = qkv.chunk(3, dim=-1)
#         out = self.attention(q , k , v)
#         out = out.view(B, self.dim, H, W)
      
#         # out = F.interpolate(x, size=( H , W ), mode='bilinear')

#         # print("out1" , out.shape)
        
#       # we need to billinear intterpolate before append
 
#         y = self.proj(out)

#         y = self.local_mod(y)
        
#         return y
        





# # class EfficientViTBackboneLarge(nn.Module):
# #     def __init__(self,
# #                 width : list[int] ,
# #                 depth : list[int] ,
# #                 block_list : list[str] or None = None,
# #                 expand_list: list[float] = [1, 4, 4, 4, 6],
# #                 in_channels=3,
# #                 fewer_norm_list: list[bool] or None = [False, False, False, True, True],
# #                 qkv_dim=32,
# #                 norm=nn.BatchNorm2d,
# #                 activation="gelu",
            
# #                 ):
# #         super().__init__()

# #         block_list = block_list or ["res", "fmb", "fmb", "mb", "att"]
# #         self.width_list = []
# #         self.stages = [] #this is the main model lineup

# #         stage0 = [
# #             ConvLayer(
# #                 in_channels=3,
# #                 out_channels=width[0],
# #                 stride=2,
# #                 norm=norm,
# #                 activation=activation,
# #             )
# #         ]

# #         #here more blocks are added to the stage0

# #         for _ in range(depth[0]):
# #             include_stage0 = self.make_block(
# #                 block= block_list[0],
# #                 in_channels=width[0],
# #                 out_channels=width[0],
# #                 stride=1,
# #                 expand_ratio=expand_list[0],
# #                 norm=norm,
# #                 activation=activation,
# #                 fewer_norm=fewer_norm_list[0],
# #             )
# #             stage0.append(ResidualBlock(include_stage0, IdentityLayer()))
# #         self.stages.append(OpSequential(stage0))
            
# #             #now we make multiple more stages
# #         in_channels = width[0]
       
# #         for stage_id, (w,d) in enumerate(zip(width[1:] ,depth[1:] ), start = 1):
# #             #make a block
           
# #             stage = []
# #             # print(block_list[stage_id])
# #             block = self.make_block(
# #                 block="mb" if block_list[stage_id] not in ["mb", "fmb"] else block_list[stage_id],
# #                 in_channels=in_channels,
# #                 out_channels=w,
# #                 stride=2,
# #                 expand_ratio=expand_list[stage_id] * 4,
# #                 norm=norm,
# #                 activation=activation,
# #                 fewer_norm=fewer_norm_list[stage_id],
# #             )
# #             stage.append(ResidualBlock(block, None))
# #             # print(w)
# #             in_channels = w
# #             for _ in range(d):
# #                 #if att make a EffViTblock else MBConv
# #                 if block_list[stage_id].startswith("att"):
# #                     stage.append(
# #                         EfficientViTModule(
# #                             in_channels=in_channels,
# #                             dimention=qkv_dim,
# #                             expand_ratio=expand_list[stage_id],
# #                             scales=(3) if block_list[stage_id] == "att@3" else (5,),
# #                             norm=(norm,norm),
# #                             activation=activation,
# #                         )
# #                     )
# #                 else:
# #                     block = self.make_block(
# #                         block=block_list[stage_id],
# #                         in_channels=in_channels,
# #                         out_channels=in_channels,
# #                         stride=1,
# #                         expand_ratio=expand_list[stage_id],
# #                         norm=norm,
# #                         activation=activation,
# #                         fewer_norm=fewer_norm_list[stage_id],
# #                     )
# #                     block = ResidualBlock(block, IdentityLayer())
# #                     stage.append(block)
# #                     # print("stage" , stage)
# #             self.stages.append(OpSequential(stage))
# #         self.stages = nn.ModuleList(self.stages)
# #     @staticmethod
# #     def make_block(
# #         block: str,
# #         in_channels: int,
# #         out_channels: int,
# #         stride: int,
# #         expand_ratio: float,
# #         norm: str,
# #         activation: str,
# #         fewer_norm: bool = False,
# #         ) -> nn.Module:
# #         if block == "res":
# #             block = ResBlock(
# #                 in_channels=in_channels,
# #                 out_channels=out_channels,
# #                 stride=stride,
# #                 use_bias=(True, False) if fewer_norm else False,
# #                 norm=(None, norm) if fewer_norm else norm,
# #                 activation=(activation, None),
# #             )
# #         elif block == "fmb":
# #             print("its FNB")
# #             block = FusedMobileBottleneckConvLayer(
# #                 in_channels=in_channels,
# #                 out_channels=out_channels,
# #                 stride=stride,
# #                 expand_ratio=expand_ratio,
# #                 use_bias=(True, False) if fewer_norm else False,
# #                 norm=(None, norm) if fewer_norm else norm,
# #                 activation=(activation, None),
# #             )
# #         elif block == "mb":
# #             block = MobileBottleneckConvLayer(
# #                 in_channels=in_channels,
# #                 out_channels=out_channels,
# #                 stride=stride,
# #                 expand_ratio=expand_ratio,
# #                 use_bias=(True, True, False) if fewer_norm else False,
# #                 norm=(None, None, norm) if fewer_norm else norm,
# #                 activation=(activation, activation, None),
# #             )
# #         else:
# #             raise ValueError(block)
# #         return block

# #     def forward(self, x: torch.Tensor) :
# #         output_dict = {"input": x}
# #         for stage_id, stage in enumerate(self.stages):
# #             output_dict["stage%d" % stage_id] = x = stage(x)
# #         output_dict["stage_final"] = x
# #         return output_dict


# # def efficientvit_backbone_l1(**kwargs) -> EfficientViTBackboneLarge:
# #     backbone = EfficientViTBackboneLarge(
# #         width=[32, 64, 128, 256, 512],
# #         depth=[1, 1, 1, 6, 6],
# #     )
# #     return backbone

NameError: name 'nn' is not defined

In [83]:
# # mkaing the complete model from scratch efficientVIT.l1

# def make_efficientViTl1():

#     stage = []

#     conv0 = 


In [84]:
# class EfficientViTBackboneLarge(nn.Module):
#     def __init__(self,
#                 width=[32, 64, 128, 256, 512],
#                 depth :  depth=[1, 1, 1, 6, 6] ,
                
#                 expand_list: list[float] = [1, 4, 4, 4, 6],
#                 in_channels=3,
#                 fewer_norm_list: list[bool] or None = [False, False, False, True, True],
#                 qkv_dim=32,
#                 norm=nn.BatchNorm2d,
#                 activation="gelu",
            
#                 ):
#         self.stage = []

#         self.stem = [
            
#         ]

#         conv0 = self.make_block(
#                 block = 'conv',
#                 in_channels=3,
#                 out_channels=width[0],
#                 stride=2,
#                 norm=norm,
#                 activation=activation,
#             )

#         res0 = stage.append(ResidualBlock(self.make_block('res',in_channels =width[0], 
#                                                           out_channels=width[0] , 
#                                                           stride=1,
#                                                           expand_ratio=expand_list[0],
#                                                           norm=norm,
#                                                           activation=activation,
#                                                           fewer_norm=fewer_norm_list[0],), IdentityLayer()))

#         self.stem.append(conv0)
#         self.stem.append(res0)
        
#         #STEM DONE

#         #woking on VITblocks
        
        

#         self.EfficintViTModules = []

        

#         #efficentvitmodule1

#         EfficintViTModule0 = self.make_res_blocks( [[block = 'fmbconv' , in_channels=width[0] ,out_channels= width[1], stride = 2 ,expand_ratio=expand_list[1]*4,scales=(3),norm=(norm,norm),
#                             activation=activation ],
#                                                     [block = 'fmbconv' ,in_channels=width[1] ,out_channels= width[1],stride = 1,expand_ratio=expand_list[1],scales=(3),norm=(norm,norm),
#                             activation=activation ]])

#         EfficintViTModule1 = self.make_res_blocks([[block = 'fmbconv' , in_channels=width[1] ,out_channels= width[2], stride = 2 ,expand_ratio=expand_list[1]*4,scales=(3),norm=(norm,norm),
#                             activation=activation ],
#                                                     [block = 'fmbconv' ,in_channels=width[2] ,out_channels= width[2],stride = 1,expand_ratio=expand_list[1],scales=(3),norm=(norm,norm),
#                             activation=activation ]])

#         EfficintViTModule2 = self.make_res_blocks([[block = 'mbconv' , in_channels=width[2] ,out_channels= width[3], stride = 2 ,expand_ratio=expand_list[2],scales=(3),norm=(norm,norm),
#                             activation=activation ],
#                                                     [block = 'mbconv' ,in_channels=width[3] ,out_channels= width[3],stride = 1,expand_ratio=expand_list[2],scales=(3),norm=(norm,norm),
#                             activation=activation ],
#                                                     [block = 'mbconv' ,in_channels=width[3] ,out_channels= width[3],stride = 1,expand_ratio=expand_list[2],scales=(3),norm=(norm,norm),
#                             activation=activation ],
#                                                     [block = 'mbconv' ,in_channels=width[3] ,out_channels= width[3],stride = 1,expand_ratio=expand_list[2],scales=(3),norm=(norm,norm),
#                             activation=activation ],
#                                                     [block = 'mbconv' , in_channels=width[2] ,out_channels= width[3], stride = 1 ,expand_ratio=expand_list[2],scales=(3),norm=(norm,norm),
#                             activation=activation ],
#                                                     [block = 'mbconv' ,in_channels=width[3] ,out_channels= width[3],stride = 1,expand_ratio=expand_list[2],scales=(3),norm=(norm,norm),
#                             activation=activation ],
#                                                     [block = 'mbconv' ,in_channels=width[3] ,out_channels= width[3],stride = 1,expand_ratio=expand_list[2],scales=(3),norm=(norm,norm),
#                             activation=activation ],])

#         EfficintViTModule3 = self.make_res_blocks([block = 'mbconv' , in_channels=width[3] ,out_channels= width[4], stride = 2 ,expand_ratio=expand_list[4]*4,scales=(3),norm=(norm,norm),
#                                 activation=activation ])

        
        
#         att1 = self.make_block('eff' ,in_channels =width[4], dimention=qkv_dim, expand_ratio = expand_list[4] , scales=(3),
#                                 norm=(norm,norm),
#                                 activation=activation, )
#         att2 = self.make_block('eff' ,in_channels =width[4], dimention=qkv_dim, expand_ratio = expand_list[4] , scales=(3),
#                                 norm=(norm,norm),
#                                 activation=activation, )
        
#         EfficintViTModule3.extend([att1 , att1 ,att1 ,att1 ,att1 ,att1 ])
#         self.EfficintViTModules.extend([EfficintViTModule0,EfficintViTModule1,EfficintViTModule2,EfficintViTModule3 ])

#         #classifier head 
#         classifier = self.make_block(
#                 block = 'conv'
#                 in_channels=width[4],
#                 out_channels=width[4]*6,
#                 stride=1,
#                 norm=norm,
#                 activation=activation,
#             )
        
#         pool = torch.nn.AdaptiveAvgPool2d(pool_type=avg, flatten=Flatten(start_dim=1, end_dim=-1)
#         classifier.append(pool)

#         seq = torch.nn.Sequential(
#             LinearLayer(width[4]*6 , 3200 , 0 , use_bias = False),
#             LinearLayer(3200 , 1000 , 0 , use_bias = False , None , None)
#         )
#         classifier.append(seq)

#         #make model

#         self.stage.append(self.stem)
#         self.stage.append(self.EfficintViTModules)
#         self.stage.append(self.classifier)
#         return self.stage

#     def make_block(self,**kwargs):
#         if block == "conv":
#             blc = ConvLayer(
#                     in_channels=in_channels,
#                     out_channels=out_channels,
#                     stride=stride,
#                     norm= None,
#                     use_bias=False
#                     activation=None,
#                 )
#         elif block == "res":
#             blc = ResBlock(
#                     in_channels=in_channels,
#                     out_channels=out_channels,
#                     stride=stride,
#                     use_bias=(True, False) if fewer_norm else False,
#                     norm=(None, norm) if fewer_norm else norm,
#                     activation=(activation, None),
#                 )
        
#         elif block == "fmb":
           
#             blc = FusedMobileBottleneckConvLayer(
#                 in_channels=in_channels,
#                 out_channels=out_channels,
#                 stride=stride,
#                 expand_ratio=expand_ratio,
#                 use_bias=(True, False) if fewer_norm else False,
#                 norm=(None, norm) if fewer_norm else norm,
#                 activation=(activation, None),
#             )
#         elif block == "mb":
#             blc = MobileBottleneckConvLayer(
#                 in_channels=in_channels,
#                 out_channels=out_channels,
#                 stride=stride,
#                 expand_ratio=expand_ratio,
#                 use_bias=(True, True, False) if fewer_norm else False,
#                 norm=(None, None, norm) if fewer_norm else norm,
#                 activation=(activation, activation, None),
#             )
#         elif block == 'eff':
#             blc = EfficientViTModule(
#                     in_channels=in_channels,
#                     dimention=qkv_dim,
#                     expand_ratio=expand_ratio,
#                     scales=(3),
#                     norm=(norm,norm),
#                     activation=activation,
#                             )
#         return blc

#     def make_modules(blocks , residual = True):
#         all = []
#         for i in range(len(block)):
#             all.append(ResidualBlock(self.make_block(blocks[i]), IdentityLayer())))
#         return all